# MCMC2.0: Iterators, Generators, and Channels

Iterators and generators are two important tools in the modern functional programming and Julia implements new "channels." I will show a typical implementation of them for MCMC.

## Iterators

Iterators are often used to save memories. If we wish to do something to each element of a large list, we do not have to store all the elements between the processes. Iterators return each element one by one, so we can do opearations one by one and each time we can discard old data to save memories. Actually, we already used iterators in usual "for" loops.

In [1]:
iter = 1 : 2

1:2

This is already iterator counting from 1 to 2.

In [2]:
element, state = iterate(iter)

(1, 1)

This is the initial state of the iterator "1:2".

In [3]:
element, state = iterate(iter, state)

(2, 2)

These are the next item and the next state.

In [4]:
result = iterate(iter, state)
result == nothing

true

If there is nothing to iterate, they return nothing.

In [5]:
iter = 1 : 5
for i in iter
    println(i)
end

1
2
3
4
5


This is the same as follow:

In [6]:
iter_result = iterate(iter)
while iter_result !== nothing
    (i, state) = iter_result
    println(i)
    iter_result = iterate(iter, state)
end

1
2
3
4
5


If we use an iterator instead of a list, we do not need to store the whole list anymore.

In [7]:
list = collect(iter)
for i in 1 : length(list)
    println(list[i] ^ 2)
end

1
4
9
16
25


This is meaningless and should be rewritten as:

In [8]:
for i in iter
    println(i ^ 2)
end

1
4
9
16
25


In this way, we can (trivially) avoid allocating a memory for the list. Or this works if the memory does not matter:

In [9]:
for i in list
    println(i ^ 2)
end

1
4
9
16
25


## IterTools

~ under construction ~

In [10]:
using IterTools
using Distributions
function Gibbs1(a::Float64, b::Float64, c::Float64)::Function
    xy::Tuple{Float64, Float64} -> begin
        x = rand(Normal(b / a * xy[2], 1 / sqrt(a)))
        x, rand(Normal(b / c * x, 1 / sqrt(c)))
    end
end
samples = iterated(Gibbs1(1.0, 0.8, 1.0), (0.0, 0.0))
collect(Iterators.take(samples, 10))

10-element Array{Tuple{Float64,Float64},1}:
 (0.0, 0.0)                                 
 (-0.7547258171926915, -0.04463370063202288)
 (-0.7313294103406582, -1.1513061740718484) 
 (-0.40359016310851203, 1.208214080768851)  
 (1.3012825498163951, 0.84780636759954)     
 (1.2701358580384523, 0.5620723701125009)   
 (1.8725968685864025, 0.7950092553589843)   
 (-0.3618188612246286, -0.6500935422375311) 
 (1.5986970302429275, 1.567748244429648)    
 (0.29538895240462004, 0.9273764102417205)  

## Generator 1 (generator expression)

An iterator is most useful when combined with a generator. We generally call something creating an iterator a generator. Generators in general include a generator expression, a generator function, etc. Julia supports a generator expression as follows:

In [12]:
gene = (x^2 for x in 1 : 5)

Base.Generator{UnitRange{Int64},getfield(Main, Symbol("##25#26"))}(getfield(Main, Symbol("##25#26"))(), 1:5)

This is very similar to a list comprehension:

In [13]:
list = [x^2 for x in 1 : 5]

5-element Array{Int64,1}:
  1
  4
  9
 16
 25

The list comprehension will allocate a memory for the whole list. The generator will save your memory.

In [14]:
for i in gene
    println(i)
end

1
4
9
16
25


## Generator 2 (generator function = resumable function)

An easier, simpler, and most useful way to create an iterator/generator is a generator function, and Python and most modern language support this type of functions. Julia does not natively support Python-type generator functions, but simply you can call ResumableFunctions.jl. This is the easiest way to reproduce the behavior of generator functions.

In [14]:
using ResumableFunctions
@resumable function Ising()::Vector{Int64}
    N = 8
    σ = ones(Int64, N)
    β = 1.0
    while true
        for i in 1 : 1000
            j = rand(1 : N)
            ΔβE = 2β * σ[j] * (σ[mod1(j + 1, N)] + σ[mod1(j - 1, N)])
            -ΔβE > log(rand()) && (σ[j] = -σ[j])
        end
        @yield σ
    end
end

Ising (generic function with 1 method)

With @resumable macro, you can define a resumable function. This function does not have "return," but yield what you want at @yield points. This function directely returns an iterater, i.e. a function iterate(iter) is already defined when we call iter = Ising(). Then, this iterater gives you the "yield" repeatedly each time you reached @yield point.

In [15]:
iter = Ising()
@show iter();

iter() = [-1, -1, -1, -1, -1, -1, -1, -1]


In [16]:
for i in 1 : 10
    println(iter())
end

[-1, 1, 1, 1, -1, -1, -1, -1]
[-1, -1, -1, 1, 1, 1, 1, -1]
[1, 1, 1, -1, -1, 1, 1, 1]
[-1, -1, -1, -1, -1, -1, -1, -1]
[1, 1, 1, -1, 1, 1, 1, 1]
[-1, -1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1]
[-1, 1, 1, 1, 1, -1, -1, -1]
[1, 1, 1, 1, 1, 1, 1, 1]
[-1, -1, -1, 1, 1, 1, -1, -1]


ResummableFunctions are very fast and memory-efficient because the macro @resummable will rewrite a function into a normal iterator directly before the JIT compilation.

## Channels

Instead of a generator function, Julia natively supports Channels for parallel computing. Especially, Channel is a stronger and more general tool to create an iterator from the function. This is a generalized concept of a generator function, but in most cases ResumableFunctions.jl is faster and memory-efficient. However, if you need a postprocess based on lazy evaluation, Channels work better from my experience.

In [17]:
function Gibbs2(a::Float64, b::Float64, c::Float64)::Channel
    Channel(ctype = Tuple{Float64,Float64}) do channel::Channel{Tuple{Float64,Float64}}
        N = 10
        x = 0.0
        y = 0.0
        put!(channel, (x, y))
        for i in 1 : N
            x = rand(Normal(b / a * y, 1 / sqrt(a)))
            y = rand(Normal(b / c * x, 1 / sqrt(c)))
            put!(channel, (x, y))
        end
    end
end

Gibbs2 (generic function with 1 method)

In [18]:
for z in Gibbs2(1.0, 0.8, 1.0)
    println(z)
end

(0.0, 0.0)
(0.2268567493797212, 2.2422260163825496)
(1.1485020750754908, -0.7486311856625307)
(0.002686751808946064, -1.0382076597118157)
(-1.9145612281517903, -1.4830879601089253)
(-2.3494083473002876, -1.3173422391171203)
(-0.1892178119415333, -0.5202011319404474)
(0.2646339889945905, -0.5069191433286592)
(0.24526114403502353, 0.7887209085939816)
(-0.14727102108047374, 1.3011692792129939)
(0.32374732418044505, -0.4803394550252035)


This code is acceptable if the number of tasks is just one.

~ under construction ~

In [19]:
function xupdate(a::Float64, b::Float64, c::Float64, input::Channel{Tuple{Float64,Float64}}, output::Channel{Tuple{Float64,Float64}})
    N = 10
    x = 0.0
    y = 0.0
    for i in 1 : N
        (x, y) = take!(input)
        x = rand(Normal(b / a * y, 1 / sqrt(a)))
        put!(output, (x, y))
    end
    close(output)
end
function yupdate(a::Float64, b::Float64, c::Float64, input::Channel{Tuple{Float64,Float64}}, output::Channel{Tuple{Float64,Float64}})
    N = 10
    x = 0.0
    y = 0.0
    for i in 1 : N
        (x, y) = take!(input)
        y = rand(Normal(b / c * x, 1 / sqrt(c)))
        println((x, y))
        put!(output, (x, y))
    end
    close(output)
end
ch1 = Channel{Tuple{Float64,Float64}}(32)
ch2 = Channel{Tuple{Float64,Float64}}(32)
@async xupdate(1.0, 0.8, 1.0, ch1, ch2)
@async yupdate(1.0, 0.8, 1.0, ch2, ch1)
put!(ch1, (0.0, 0.0));

(1.290545258249347, 1.5204399797238781)
(0.21063803259475922, -0.679626885073408)
(-0.5654469441342398, -0.13967597204683474)
(0.04866095529962684, 1.8856041211826469)
(0.8548483728113733, 1.0810868469527135)
(0.3227294055973985, 0.19578779196614943)
(1.4252567610503524, 1.114858695432623)
(1.1865475809567494, 0.02364702577480271)
(-0.4307690206327719, -1.1446996842244603)
(-0.15805673488157457, -0.6290038176569472)


The difference bewteen Channels and ResumableFunctions lies in how they compile a code. Check: https://white.ucc.asn.au/2017/11/18/Lazy-Sequences-in-Julia.html

## RemoteChannels

It will be discussed in MCMC4.0.

## Lazy

Lazy.jl is sometimes useful.

In [20]:
using Lazy
takewhile(x -> x < 3, 1 : 5)

(1 2)

However, the coding style of Lazy.jl is different from ordinarly Julia, so I won't use this package unless it is necessary.

**Exercise**: implement takewhile and dropwhile for iterators. These functions are sometimes useful for MCMC.